In [1]:
import pandas as pd
import nltk
from pprint import pprint
import pickle
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score
# from sklearn.svm import LinearSVC
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# import xgboost as xgb
# from sklearn.svm import SVC
# import matplotlib.pyplot as plt
# from sklearn.neural_network import MLPClassifier
# from sklearn.decomposition import TruncatedSVD
# from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
# from sklearn import pipeline
# from sklearn import metrics
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.backend as K
from keras.preprocessing import sequence, text
from keras.layers.embeddings import Embedding
from keras.layers import SpatialDropout1D, LSTM
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
tqdm.pandas()

In [2]:
# from fastai import *
# from fastai.text import *

In [3]:
# # bs=48
# bs=16
# #bs=192
# torch.cuda.set_device(0)

In [4]:
#!/data/home/manokuma/.local/bin/kaggle 
# import os
# os.environ['KAGGLE_USERNAME'] = 'hackmano'
# os.environ['KAGGLE_KEY'] = ''
# !/data/home/manokuma/.local/bin/kaggle competitions download -c nlp-getting-started
# !unzip glove.6B.zip

## Reading and Understanding data

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [59]:
trainer,valid = train_test_split(train, random_state=42, test_size=0.2, shuffle=True)

In [4]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [5]:
sentences = train["text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 7613/7613 [00:00<00:00, 250716.37it/s]

{'crushed.': 1, 'grandpa': 2, '@Change': 9, 'http://t.co/BDsgF1CfaX': 1, 'shift': 10}


In [29]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:
            try:
                a[word] = embeddings_index[word.lower()]
                k += vocab[word]
            except:
                oov[word] = vocab[word]
                i += vocab[word]
                pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [84]:
pprint(test.text.values[210])

('@Sunnyclaribel. how many of @DPJHodges articles just attack labour or defend '
 'conservatives? see for yourself: http://t.co/shAAIjO2ZC')


In [30]:
oov = check_coverage(vocab,word_embeddings)

100%|██████████| 26234/26234 [00:00<00:00, 773831.14it/s]

Found embeddings for 64.20% of vocab
Found embeddings for  94.00% of all text


In [26]:
oov[:20]

[('I', 1601),
 ('The', 586),
 ('A', 317),
 ('In', 161),
 ('It', 140),
 ('You', 117),
 ('California', 117),
 ('News', 112),
 ('S', 111),
 ('RT', 110),
 ('This', 110),
 ('To', 109),
 ('We', 108),
 ('New', 103),
 ('My', 98),
 ('If', 95),
 ('YouTube', 92),
 ('Hiroshima', 90),
 ('THE', 86),
 ('Is', 86)]

In [55]:
'921' in word_embeddings

True

In [6]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [69]:
yolo = take(20, word_embeddings.keys())

In [70]:
pprint(yolo)

['neitz',
 'lawter',
 'haz',
 'cytyc',
 'kabayama',
 'preclusive',
 'jiazheng',
 'adriatic',
 '176.5',
 'spitze',
 'merita',
 'lightweights',
 'jansons',
 'dorcus',
 'somatoform',
 'metabolizing',
 '34-foot',
 '0.997',
 'bauch',
 'foot-long']


In [20]:
#https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings
def clean_text(x):
    x = str(x)
    for punct in '&/-?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, ' {0} '.format(punct))
    return x

In [22]:
train["text"] = train["text"].progress_apply(lambda x: clean_text(x))
sentences = train["text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

100%|██████████| 7613/7613 [00:00<00:00, 212592.87it/s]


In [8]:
word_embeddings = {}
with open('word_embeddings.pickle', 'rb') as g:
    word_embeddings = pickle.load(g)

False

In [56]:
stop_words = set(stopwords.words('english')) 
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    #words = [w for w in words if not w in stop_words]
    #words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(word_embeddings[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if (type(v) != np.ndarray):
        return np.zeros(300)
    return v/np.sqrt((v ** 2).sum())

In [57]:
hell = sent2vec("MY first sentence in Python is not Hello world")


In [10]:

# word_embeddings = {}
# f = open('glove.6B.300d.txt', encoding='utf8')
# for line in tqdm(f):
#     values = line.split()  
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     word_embeddings[word] = coefs
# f.close()
# print('Found %s word vectors.' % len(word_embeddings))


In [11]:
# with open('word_embeddings.pickle', 'wb') as g:
#     pickle.dump(word_embeddings,g)

In [12]:
#nltk.download('stopwords')
#nltk.download('punkt')


In [82]:
ytrain_enc = np_utils.to_categorical(trainer.target.values)
yvalid_enc = np_utils.to_categorical(valid.target.values)

In [60]:
xtrain_glove = [sent2vec(x) for x in tqdm(trainer.text.values)]
xvalid_glove = [sent2vec(x) for x in tqdm(valid.text.values)]
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

100%|██████████| 1523/1523 [00:00<00:00, 2666.07it/s]


In [61]:
test_glove = [sent2vec(x) for x in tqdm(test.text.values)]
test_glove = np.array(test_glove)

100%|██████████| 3263/3263 [00:00<00:00, 3569.47it/s]


In [91]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 300


token.fit_on_texts(list(trainer.text.values) + list(valid.text.values))
xtrain_seq = token.texts_to_sequences(trainer.text.values)
xvalid_seq = token.texts_to_sequences(valid.text.values)
test_seq = token.texts_to_sequences(test.text.values)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len, padding='post')
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len, padding='post')
test_pad = sequence.pad_sequences(test_seq, maxlen=max_len, padding='post')

word_index = token.word_index

In [93]:
#xtrain_glove[:3]

In [ ]:
yolo = take(20, word_index.items())
yolo

In [94]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 21952/21952 [00:00<00:00, 467697.97it/s]


In [73]:
#train.head()
#train["text"][1000]
#word_embeddings.get("hello")

In [20]:
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


### Try k-nearest neighbour on the data

<h2>Use tf-idf on the text</h2>

In [40]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [41]:
tfv.fit(list(trainer.text.values) + list(valid.text.values))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words='english', strip_accents='unicode',
                sublinear_tf=1, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=1, vocabulary=None)

In [42]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [43]:
xtrain_tfv =  tfv.transform(trainer.text.values) 
xvalid_tfv = tfv.transform(valid.text.values)
pprint(vars(xtrain_tfv))

{'_shape': (6090, 9229),
 'data': array([0.33437892, 0.25069802, 0.06115396, ..., 0.31132424, 0.33215319,
       0.26934142]),
 'indices': array([8605, 8604, 7909, ..., 2824,  525,  524], dtype=int32),
 'indptr': array([    0,    15,    21, ..., 76789, 76810, 76825], dtype=int32),
 'maxprint': 50}


<h2>Use Logistic Regression and linear svc on data</h2>

In [108]:
#clf = KNeighborsClassifier(n_neighbors=5)
#help(LogisticRegression)
clf = LogisticRegression(C=2)
clf.fit(xtrain_tfv, trainer.target)
print("Number of features used:", np.sum(clf.coef_ != 0))

Number of features used: 9193


In [109]:
#predictions = clf.predict_proba(xvalid_tfv)
train_predictions =  clf.predict(xtrain_tfv)
predictions =  clf.predict(xvalid_tfv)
print("Test set predictions:", predictions)

Test set predictions: [0 0 0 ... 1 1 1]


In [110]:
print ("f1_score on training: %0.3f " % f1_score(trainer.target.values, train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, predictions))

f1_score on training: 0.870 
f1_score on test: 0.744 


In [61]:
print("Training set accuracy: {:.2f}".format(clf.score(xtrain_tfv, trainer.target)))
print("Test set accuracy: {:.2f}".format(clf.score(xvalid_tfv, valid.target)))

Training set accuracy: 0.93
Test set accuracy: 0.78


In [59]:
train_predictions = clf.predict(xtrain_tfv)
print ("f1_score on training: %0.3f " % f1_score(trainer.target.values, train_predictions))

f1_score on training: 0.916 


In [60]:
#print ("logloss: %0.3f " % multiclass_logloss(valid.target.values, predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, predictions))
#f1 score should be made as close to 1 as possible

f1_score on test: 0.727 


In [28]:
valid.target.values.shape[0]

1523

In [74]:
#help(LinearSVC)
sv_clf = LinearSVC(C=0.38)
sv_clf.fit(xtrain_tfv, trainer.target)

LinearSVC(C=0.38, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [75]:
sv_predictions = sv_clf.predict(xvalid_tfv)
sv_train_predictions = sv_clf.predict(xtrain_tfv)


In [76]:
print ("f1_score on training: %0.3f " % f1_score(trainer.target.values, sv_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, sv_predictions))

f1_score on training: 0.896 
f1_score on test: 0.741 


<h2>Use Naive bayes on the data</h2>

In [55]:
nb_clf = MultinomialNB(alpha=1)
nb_clf.fit(xtrain_tfv, trainer.target)


MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [56]:
nb_predictions = nb_clf.predict(xvalid_tfv)
nb_train_predictions = nb_clf.predict(xtrain_tfv)

In [57]:
print ("f1_score on training: %0.3f " % f1_score(trainer.target.values, nb_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, nb_predictions))

f1_score on training: 0.816 
f1_score on test: 0.726 


<h1> Using Count vectorizer instead of td-idf below<h1>

In [34]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

In [35]:
ctv.fit(list(trainer.text.values) + list(valid.text.values))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [36]:
xtrain_ctv = ctv.transform(trainer.text.values)
xvalid_ctv = ctv.transform(valid.text.values)

In [121]:
clf_ctv = LogisticRegression(C=0.2)
clf_ctv.fit(xtrain_ctv, trainer.target)
print("Number of features used:", np.sum(clf_ctv.coef_ != 0))

Number of features used: 108907


In [122]:
ctv_train_predictions = clf_ctv.predict(xtrain_ctv)
ctv_predictions = clf_ctv.predict(xvalid_ctv)


In [123]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, ctv_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, ctv_predictions))

f1_score on train: 0.719 
f1_score on test: 0.619 


In [30]:
dt_clf = DecisionTreeClassifier(max_depth = 75)
dt_clf.fit(xtrain_tfv, trainer.target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=75, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [31]:
dt_train_predictions = dt_clf.predict(xtrain_tfv)
dt_predictions = dt_clf.predict(xvalid_tfv)

In [32]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, dt_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, dt_predictions))

f1_score on train: 0.868 
f1_score on test: 0.650 


In [19]:
#help(RandomForestClassifier)
forest = RandomForestClassifier()
forest.fit(xtrain_tfv, trainer.target.values)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
rfc_train_predictions = forest.predict(xtrain_tfv)
rfc_predictions = forest.predict(xvalid_tfv)

In [23]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, rfc_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, rfc_predictions))

f1_score on train: 0.983 
f1_score on test: 0.700 


|Classifier|Data Transformer|Parameters|f1_Score_training|f1_Score_test|
| --- | --- | --- | --- | --- |
|LogisticRegression|tf-idf|C=2|0.870|0.744 
|LinearSVC|tf-idf|C=0.38|0.896|0.741| 
|LinearSVC|tf-idf|C=0.1|0.836|0.734| 
|MultinomialNB|tf-idf|default(alpha=1)|0.816|0.726|
|LogisticRegression|countVectorizer|liblinear/l1|0.857|0.738|
|DecisionTreeClassifier|tf-idf|Default|0.983|0.656|
|RandomForestClassifier|tf-idf|Default|0.983|0.700|
|GradientBoostingClassifier|tf-idf|lr = 1|0.87|0.700|
|xgboost|tf-idf|lr=0.3,nestimators=200,max_depth=5|0.856|0.718|
|SVC|tf-idf|default|0.942|0.739|
|MLP|tf-idf|max_iter=30,lbfgs,random_state=42|0.869|0.736|
|XGB|glove|default|-|0.76|
|LogisticRegression|glove|default|0.769|0.771|
|MLP|glove|max_iter=50,adam|0.853|0.775|
|Dense|glove,standardScaler|adam|0.85|0.805|
|LSTM|glove,tokenize_padding|adam|0.9397|0.829|

In [35]:
gbc_clf = GradientBoostingClassifier(learning_rate = 1)
gbc_clf.fit(xtrain_tfv, trainer.target.values)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [38]:
gbc_train_predictions = gbc_clf.predict(xtrain_tfv)
gbc_predictions = gbc_clf.predict(xvalid_tfv)

In [40]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, gbc_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, gbc_predictions))

f1_score on train: 0.877 
f1_score on test: 0.690 


In [35]:
#help(xgb)
xgb_clf = xgb.XGBClassifier(max_depth = 5, n_estimators = 200, learning_rate = 0.1, nthread = 8)

In [36]:
xgb_clf.fit(xtrain_tfv, trainer.target.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [37]:
xgb_train_predictions = xgb_clf.predict(xtrain_tfv)
xgb_predictions = xgb_clf.predict(xvalid_tfv)

In [38]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, xgb_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, xgb_predictions))

f1_score on train: 0.774 
f1_score on test: 0.701 


In [43]:
#help(SVC)
svc_clf = SVC()
svc_clf.fit(xtrain_tfv, trainer.target.values)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [44]:
svc_train_predictions = svc_clf.predict(xtrain_tfv)
svc_predictions = svc_clf.predict(xvalid_tfv)

In [45]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, svc_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, svc_predictions))

f1_score on train: 0.942 
f1_score on test: 0.739 


In [28]:
f1_scorer = metrics.make_scorer(f1_score, greater_is_better=True, needs_proba=False)
svd = TruncatedSVD()
scl = preprocessing.StandardScaler()
lr_model = LogisticRegression()
clf = pipeline.Pipeline([('svd', svd),
                        ('scl', scl),
                        ('lr', lr_model)])

In [22]:
param_grid = {'svd__n_components' : [120,180],
             'lr__C':[0.1, 1.0, 10],
             'lr__penalty': ['l1','l2']}

In [29]:
model = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = f1_scorer, verbose = 10,n_jobs=-1,iid=True,
                    refit = True, cv=2)
model.fit(xtrain_tfv, trainer.target.values)
print("Best score:%0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:   14.7s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   17.0s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:825: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Best score:0.689
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 180


# #Applying Multilayer Perceptron

In [33]:
#help(MLPClassifier)
mlp = MLPClassifier(max_iter=30, solver='lbfgs')
mlp.fit(xtrain_tfv, trainer.target.values)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=30,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [34]:
mlp_train_predictions = mlp.predict(xtrain_tfv)
mlp_predictions = mlp.predict(xvalid_tfv)

In [35]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, mlp_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, mlp_predictions))

f1_score on train: 0.918 
f1_score on test: 0.714 


# Using Glove for Word Embeddings

In [13]:
xgb_clf_glove = xgb.XGBClassifier(nthread=10, silent = False, max_depth=10, n_estimators=200, learning_rate=0.1)
xgb_clf_glove.fit(xtrain_glove, trainer.target.values)
xgb_glove_predictions = xgb_clf_glove.predict(xvalid_glove)

NameError: name 'xgb' is not defined

In [61]:
#print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, xgb_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, xgb_glove_predictions))

f1_score on test: 0.769 


In [71]:

#help(MLPClassifier)
mlp_glove = MLPClassifier(max_iter=70, solver='adam')
mlp_glove.fit(xtrain_glove, trainer.target.values)
#lr_clf_glove = LinearSVC(C=0.38)
#lr_clf_glove.fit(xtrain_glove, trainer.target.values)
mlp_glove_predictions = mlp_glove.predict(xvalid_glove)
mlp_glove_train_predictions = mlp_glove.predict(xtrain_glove)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [46]:
key, val = next(iter(word_embeddings.items()))
print(key)

the


In [72]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, mlp_glove_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, mlp_glove_predictions))

f1_score on train: 0.890 
f1_score on test: 0.759 


## Using Neural network

In [76]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [20]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[f1])

In [21]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [21]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))
#help(model.fit)

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 5s 843us/step - loss: 0.6200 - f1: 0.7373 - val_loss: 0.4482 - val_f1: 0.8009
Epoch 2/5
6090/6090 [==============================] - 1s 100us/step - loss: 0.4564 - f1: 0.8056 - val_loss: 0.4242 - val_f1: 0.8035
Epoch 3/5
6090/6090 [==============================] - 1s 98us/step - loss: 0.3972 - f1: 0.8272 - val_loss: 0.4372 - val_f1: 0.8050
Epoch 4/5
6090/6090 [==============================] - 1s 100us/step - loss: 0.3687 - f1: 0.8455 - val_loss: 0.4445 - val_f1: 0.8014
Epoch 5/5
6090/6090 [==============================] - 1s 99us/step - loss: 0.3409 - f1: 0.8566 - val_loss: 0.4598 - val_f1: 0.7963


In [34]:
predictions = model.predict(xvalid_glove_scl)

ValueError: Error when checking input: expected embedding_6_input to have shape (70,) but got array with shape (300,)

In [47]:
predictions = np.where(predictions > 0.5, 1, 0)
predictions = predictions[:,0]
pprint(predictions)

array([1, 1, 1, ..., 0, 0, 1])


In [50]:
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, predictions))

f1_score on test: 0.222 


#### LSTM

In [96]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[f1])
earlystop = EarlyStopping(monitor='val_f1', min_delta=0, patience=12, verbose=0, mode='auto')

In [97]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks = [earlystop])

Train on 6090 samples, validate on 1523 samples
Epoch 1/100
6090/6090 [==============================] - 9s 1ms/step - loss: 0.6871 - f1: 0.5228 - val_loss: 0.6824 - val_f1: 0.5739
Epoch 2/100
6090/6090 [==============================] - 4s 735us/step - loss: 0.6837 - f1: 0.5693 - val_loss: 0.6822 - val_f1: 0.5739
Epoch 3/100
6090/6090 [==============================] - 5s 740us/step - loss: 0.6836 - f1: 0.5692 - val_loss: 0.6822 - val_f1: 0.5739
Epoch 4/100
6090/6090 [==============================] - 4s 737us/step - loss: 0.6839 - f1: 0.5699 - val_loss: 0.6825 - val_f1: 0.5739
Epoch 5/100
6090/6090 [==============================] - 5s 739us/step - loss: 0.6842 - f1: 0.5696 - val_loss: 0.6822 - val_f1: 0.5739
Epoch 6/100
6090/6090 [==============================] - 4s 739us/step - loss: 0.6844 - f1: 0.5696 - val_loss: 0.6826 - val_f1: 0.5739
Epoch 7/100
6090/6090 [==============================] - 4s 739us/step - loss: 0.6837 - f1: 0.5698 - val_loss: 0.6823 - val_f1: 0.5739
Epoch 8/1

In [119]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 6090 samples, validate on 1523 samples
Epoch 1/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6897 - f1: 0.5679 - val_loss: 0.7693 - val_f1: 0.4260
Epoch 2/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6906 - f1: 0.5634 - val_loss: 0.6822 - val_f1: 0.5740
Epoch 3/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6859 - f1: 0.5691 - val_loss: 0.6835 - val_f1: 0.5740
Epoch 4/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6844 - f1: 0.5725 - val_loss: 0.6841 - val_f1: 0.5740
Epoch 5/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6847 - f1: 0.5708 - val_loss: 0.6823 - val_f1: 0.5740
Epoch 6/100
6090/6090 [==============================] - 26s 4ms/step - loss: 0.6861 - f1: 0.5689 - val_loss: 0.6835 - val_f1: 0.5740
Epoch 7/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6838 - f1: 0.5724 - val_loss: 0.6834 - val_f1: 0.5740
Epoch 8/100
60

6090/6090 [==============================] - 26s 4ms/step - loss: 0.6836 - f1: 0.5715 - val_loss: 0.6822 - val_f1: 0.5740
Epoch 62/100
6090/6090 [==============================] - 26s 4ms/step - loss: 0.6835 - f1: 0.5697 - val_loss: 0.6823 - val_f1: 0.5740
Epoch 63/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6835 - f1: 0.5706 - val_loss: 0.6822 - val_f1: 0.5740
Epoch 64/100
6090/6090 [==============================] - 27s 4ms/step - loss: 0.6835 - f1: 0.5680 - val_loss: 0.6822 - val_f1: 0.5740
Epoch 65/100
6090/6090 [==============================] - 26s 4ms/step - loss: 0.6836 - f1: 0.5688 - val_loss: 0.6823 - val_f1: 0.5740
Epoch 66/100
6090/6090 [==============================] - 26s 4ms/step - loss: 0.6835 - f1: 0.5715 - val_loss: 0.6823 - val_f1: 0.5740
Epoch 67/100
6090/6090 [==============================] - 26s 4ms/step - loss: 0.6836 - f1: 0.5688 - val_loss: 0.6822 - val_f1: 0.5740
Epoch 68/100
6090/6090 [==============================] - 27s 4ms/st

In [122]:
#predictions = model.predict(xvalid_pad)
predictions = model.predict(test_pad)

In [25]:
 #with open('predictions.pickle', 'wb') as g:
     #pickle.dump(predictions,g)

In [26]:
predictions = {}
with open('predictions.pickle', 'rb') as g:
    predictions = pickle.load(g)

In [109]:
predictions = np.where(predictions > 0.5, 1, 0)
predictions = predictions[:,0]
pprint(predictions)

array([1, 1, 1, ..., 1, 1, 1])


In [123]:
predictions

array([[0.56965035, 0.43034968],
       [0.56965035, 0.43034968],
       [0.56965035, 0.43034968],
       ...,
       [0.56965035, 0.43034968],
       [0.56965035, 0.43034968],
       [0.56965035, 0.43034968]], dtype=float32)

In [103]:
#pprint(sample)

In [110]:
#solution = test.iloc[:,0:1]
#predictions = np.array(map(str, predictions))
#predictions = np.insert(predictions, 0, "target", axis=0)
#pprint(predictions[:])
final_data = {'id' : test.id,'target':predictions}
submission = pd.DataFrame(data=final_data)


In [106]:
#pprint(submission)

In [111]:
submission.to_csv('submission_lstm.csv', index=False)

In [62]:

#help(MLPClassifier)
lr_glove = LogisticRegression()
lr_glove.fit(xtrain_glove, trainer.target.values)
#lr_clf_glove = LinearSVC(C=0.38)
#lr_clf_glove.fit(xtrain_glove, trainer.target.values)
lr_glove_predictions = lr_glove.predict(xvalid_glove)
lr_glove_train_predictions = lr_glove.predict(xtrain_glove)

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [63]:
print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, lr_glove_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, lr_glove_predictions))

f1_score on train: 0.760 
f1_score on test: 0.765 


In [107]:
#print ("f1_score on train: %0.3f " % f1_score(trainer.target.values, lr_glove_train_predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, predictions))

f1_score on test: 0.598 


In [77]:
predictions = lr_glove.predict(test_glove)
pprint(predictions)
#final_data = {'id' : test.id,'target':predictions}
#submission = pd.DataFrame(data=final_data)


array([1, 1, 1, ..., 1, 1, 1])


In [78]:
final_data = {'id' : test.id,'target':predictions}
submission = pd.DataFrame(data=final_data)

In [79]:
submission.to_csv('submission_lr_glove.csv', index=False)